This notebook demonstrates how to save and load safetensors models, using Llama 2 7B for example.
More details about safetensors in this article: [Safe, Fast, and Memory Efficient Loading of LLMs with Safetensors](https://kaitchup.substack.com/p/safe-fast-and-memory-efficient-loading)

*You will need a GPU with at least 29 GB of VRAM to run this notebook (e.g., Google Colab Pro A100).*




In [ ]:
!pip install transformers accelerate
!pip install nvidia-ml-py3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=796dec06ff93d58468de4b9e6c43b96b788393372fada6375257b19866a64f06
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


Import the required libraries. I use pynml to benchmark the VRAM consumption.

In [ ]:
from transformers import AutoModelForCausalLM
from pynvml import *
import time

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

model_id = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Benchmark the loading time and memory consumption of loading Llama 2 7B safetensors. By default, transformers loads the safetensors version if it exists. It should consume less than 4 GB of CPU RAM.
*Note: You should download the model first for a fair benchmarking. Run the cell, then restart the runtime.*


In [ ]:
start_time = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_id, device_map={"": 0}
)
duration = float(time.time() - start_time)
print("--- %s seconds ---" % (round(duration,3)))
print_gpu_utilization()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- 16.745 seconds ---
GPU memory occupied: 26709 MB.


Benchmark the loading time and memory consumption of loading Llama 2 7B without safetensors. *Note: Restart the runtime before running this cell.*

In [ ]:
start_time = time.time()
model = AutoModelForCausalLM.from_pretrained(
          model_id, device_map={"": 0}, use_safetensors=False
)
duration = float(time.time() - start_time)
print("--- %s seconds ---" % (round(duration,3)))
print_gpu_utilization()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

--- 23.311 seconds ---
GPU memory occupied: 26709 MB.


Save the model to pickle format.

In [ ]:
start_time = time.time()
model.save_pretrained("llama2_PyTorch_pickle")
duration = float(time.time() - start_time)
print("--- %s seconds ---" % (round(duration,3)))
print_gpu_utilization()

--- 79.358 seconds ---
GPU memory occupied: 26709 MB.


Save the model to safetensors.

In [ ]:
start_time = time.time()
model.save_pretrained("llama2_safetensors", safe_serialization=True)
duration = float(time.time() - start_time)
print("--- %s seconds ---" % (round(duration,3)))
print_gpu_utilization()

--- 109.432 seconds ---
GPU memory occupied: 26709 MB.
